- Given u, A (n x n), v, generate Krylov sequence u^TA^iv from i = 0 to i = 2*n - 1
- Find the min poly of the sequence using Berlekamp-Massey
- Check if the Berlekamp Massey min poly of k first elements of the sequence is the min poly of the final sequence
- What types of u, A, v may influence k?

# Early Min Poly Generation from Krylov Sequence

In [22]:
from sage.matrix.berlekamp_massey import berlekamp_massey
def krylov_sequence(u, A, v, length):
    return [u.dot_product((A^i)*v) for i in range(length)]

In [23]:
def generate_fullrank_matrix(n, F):
    return random_matrix(F, n, algorithm='unimodular')
def generate_nilpotent_matrix(n, F):
    A = zero_matrix(F, n)
    for i in range(n - 1):
        A[i, i + 1] = 1
    return A

In [42]:
def experiment_minpoly_degree_effect(n=5, p=7, trials=1000):
    F = GF(p)
    print(f"Running experiment over GF({p}) with matrix size n = {n}\n")

    def run_case(matrix_label, A):
        true_poly = A.minpoly()
        deg = true_poly.degree()
        early_count = 0
        early_ks = []

        for _ in range(trials):
            u = random_vector(F, n)
            v = random_vector(F, n)
            seq = [u.dot_product((A^i) * v) for i in range(2 * n)]
            mu = berlekamp_massey(seq)
            if mu != true_poly:
                continue
            for k in range(2 * deg, 2 * n, 2):  # must be even
                if berlekamp_massey(seq[:k]) == mu:
                    early_count += 1
                    early_ks.append(k)
                    break

        rate = 100 * early_count / trials
        avg_k = sum(early_ks)/len(early_ks) if early_ks else "N/A"

        print(f"{matrix_label}")
        print(f"  Minpoly degree: {deg}")
        print(f"  Early recovery rate: {float(rate):.2f}%")
        print(f"  Avg early k: {avg_k}\n")

    A_full = generate_fullrank_matrix(n, F)
    A_nil = generate_nilpotent_matrix(n, F)

    run_case("Full-rank matrix", A_full)
    run_case("Nilpotent matrix", A_nil)

In [43]:
experiment_minpoly_degree_effect(n=100, p=1009, trials=10)

Running experiment over GF(1009) with matrix size n = 100

Full-rank matrix
  Minpoly degree: 100
  Early recovery rate: 0.00%
  Avg early k: N/A

Nilpotent matrix
  Minpoly degree: 100
  Early recovery rate: 0.00%
  Avg early k: N/A

